In [4]:
from collections import deque

def OneReachabilityGame(G, A):

    winning_player1 = set()
    losing_player1 = set()

    #Created reverse edges
    reverse_edges = {v: set() for v in G['V1'].union(G['V2'])}
    for u, neighbors in G['E'].items():
        for v in neighbors:
            reverse_edges[v].add(u)

    #accepting states (winning set for Player 1)
    queue = deque(A)

    while queue:
        current = queue.popleft()
        winning_player1.add(current)
        #If we have one edge to AS in RE from V1, then winning set, p is for predecessor edge
        for p in reverse_edges[current]:
            if p in G['V1']:
                if p not in winning_player1:
                    queue.append(p)
            else:
                # For a Player 2 vertex, all successors must be winning for it to be winning
                if all(successor in winning_player1 for successor in G['E'][p]):
                    if p not in winning_player1:
                        queue.append(p)

    return winning_player1

G = {
    'V1': {1, 2, 3},
    'V2': {4, 5, 6},
    'E': {1: {4}, 2: {5}, 3: {6}, 4: {2}, 5: {3}, 6: {1}},
    's': 1,
    'A': {3, 6}
}

WS = OneReachabilityGame(G, G['A'])
WS


{1, 2, 3, 4, 5, 6}

In [6]:
G = {
    'V1': { 2, 3, 4},
    'V2': { 1, 5, 6},
    'E': {1: {2}, 2: {5}, 3: {4}, 4: {3}, 5: {6, 3, 1}, 6: {6}},
    's':1,
    'A': {4}   
}
WS = OneReachabilityGame(G, G['A'])
WS

# 'reverse_edges': {2: {1}, 5: {2}, 4: {3}, 3: {4, 5}, 6: {5}, 6: {6}, 1:{ 5}}

{3, 4}

In [100]:
from collections import deque
def Attr1(B, G):
    """ Attractor for Player 1 """
    attractor = set(B)
    queue = deque(B)

    while queue:
        u = queue.popleft()
        if u in G['reverse_edges']:
            for v in G['reverse_edges'][u]:
                if v not in attractor:
                    if v in G['V1'] or all(w in attractor for w in G['E'][v]):
                        attractor.add(v)
                        queue.append(v)
    return attractor


def Attr2(TR, G):
    """ Attractor for Player 2 """
    attractor = set(TR)
    queue = deque(TR)

    while queue:
        u = queue.popleft()
        if u in G['reverse_edges']:
            for v in G['reverse_edges'][u]:
                if v not in attractor and v in G['V2']:
                    if any(w in attractor for w in G['E'][v]):
                        attractor.add(v)
                        queue.append(v)
    return attractor


def AvoidSetClassical(G, B):
    """ Compute the Avoid Set in the classical Buchi Game algorithm """
    Ri = Attr1(B, G)
    print('Ri', Ri)
    Tri = G['S'] - Ri
    Wi = Attr2(Tri, G)
    print('Wi',Wi)
    return Wi


def BuchiGameAlgorithm(G, B):
    """ Classical algorithm for Buchi Games """
    
    reverse_edges = {v: set() for v in G['V1'].union(G['V2'])}
    for u, neighbors in G['E'].items():
        for v in neighbors:
            reverse_edges[v].add(u)
    G['reverse_edges']=reverse_edges
    Gi = {'V1': G['V1'], 'V2': G['V2'], 'E': G['E'], 'S': G['V1'].union(G['V2']), 'reverse_edges': G['reverse_edges']}
    Si = Gi['S']
    Wi = set()
    i = 0
    print(G)
    disqualified_states = set()
    previous_disqualified_states = set()
    W = set()
    while True:
        Wi = AvoidSetClassical(Gi, B & Si)
#         if not Wi:
#             break
        #winning states of player 1
        Ri=Si-Wi
        print('Ri',Ri)
        
        for w in Ri:
            if w in Gi['V1']:
#                 print('w',w)
                if w in Gi['E'] and all(v in Wi for v in G['E'][w]):
#                     print(w)
                    
#                     for v in Gi['E'][w]:
#                         if v in Wi:
                    disqualified_states.add(w)
                    break
                if(w not in Gi['E']):
                    disqualified_states.add(w)
                    break
                
#                 else:
#                     disqualified_states.add(w)
#                     break
                    
            else:
                if w not in Gi['E']:
                    disqualified_states.add(w)
                    break
            
                    
                
        print('disqualified_states',disqualified_states)
        print('Wi',Wi)
#         for w in Ri:
#             if w in Gi['V1']:
#                 if w in Gi['E']:
# #                     print(w)
#                     for v in Gi['E'][w]:
#                         if v in Wi:
#                             disqualified_states.add(w)
#                             break
                        
#         if not disqualified_states:
#             break
        if (not disqualified_states - previous_disqualified_states) and not Wi:
            break
        

        # Update Si to exclude disqualified states
        Si = Ri - disqualified_states

        # Update the game graph for the next iteration
        Gi['S'] = Si
        W.update(Wi - disqualified_states)
        previous_disqualified_states = disqualified_states.copy()


        

        i += 1

    return Si



In [103]:
G1={
    'V1':{1,2,5}, 
    'V2':{3,4},
    'E':{1:{4,5}, 2:{3}, 4:{1,2}, 5:{1,2}}
}
B1={3,4,5}
winning_set1 = BuchiGameAlgorithm(G1, B1)
winning_set1

{'V1': {1, 2, 5}, 'V2': {3, 4}, 'E': {1: {4, 5}, 2: {3}, 4: {1, 2}, 5: {1, 2}}, 'reverse_edges': {1: {4, 5}, 2: {4, 5}, 3: {2}, 4: {1}, 5: {1}}}
Ri {1, 2, 3, 4, 5}
Wi set()
Ri {1, 2, 3, 4, 5}
disqualified_states {3}
Wi set()
Ri {1, 4, 5}
Wi {2, 4}
Ri {1, 5}
disqualified_states {3}
Wi {2, 4}
Ri {1, 5}
Wi set()
Ri {1, 5}
disqualified_states {3}
Wi set()


{1, 5}

In [105]:
G = {
    'V1': { 2, 3, 4},
    'V2': { 1, 5, 6},
    'E': {1: {1,2}, 2: {5}, 3: {4}, 4: {3}, 5: {6, 3}, 6: {6}}
    }

B = {2, 4}
winning_set = BuchiGameAlgorithm(G, B)
winning_set

{'V1': {2, 3, 4}, 'V2': {1, 5, 6}, 'E': {1: {1, 2}, 2: {5}, 3: {4}, 4: {3}, 5: {3, 6}, 6: {6}}, 'reverse_edges': {1: {1}, 2: {1}, 3: {4, 5}, 4: {3}, 5: {2}, 6: {5, 6}}}
Ri {2, 3, 4}
Wi {1, 5, 6}
Ri {2, 3, 4}
disqualified_states {2}
Wi {1, 5, 6}
Ri {3, 4}
Wi set()
Ri {3, 4}
disqualified_states {2}
Wi set()


{3, 4}

In [104]:
# Example usage
G = {
    'V1': {1, 2, 3},
    'V2': {4, 5, 6},
    'E': {1: {4}, 2: {5}, 3: {6}, 4: {2}, 5: {3}, 6: {1}},
}
B = {3, 6}

# Computing the winning set for the Büchi game
winning_set = BuchiGameAlgorithm(G, B)
winning_set


{'V1': {1, 2, 3}, 'V2': {4, 5, 6}, 'E': {1: {4}, 2: {5}, 3: {6}, 4: {2}, 5: {3}, 6: {1}}, 'reverse_edges': {1: {6}, 2: {4}, 3: {5}, 4: {1}, 5: {2}, 6: {3}}}
Ri {1, 2, 3, 4, 5, 6}
Wi set()
Ri {1, 2, 3, 4, 5, 6}
disqualified_states set()
Wi set()


{1, 2, 3, 4, 5, 6}